----
#### DataLoader

This is our data loader. Should get an index file for an experment type (9mm,5mm etc). The index file should contain a path to each indiviual training experiment and the class it belongs to high, medium, or low.
Note that the class is represented as a int, the map is shown below

|class         | numerical value|
|--------------|----------------|
|<i>high</i>   |               2|
|<i>medium</i> |               1|
|<i>low</i>    |               0|

Example index file (Prepocessing/data/spikeTrains/1.5-SpikeTrains/index.csv)
<pre>
spikeTrain_1.csv,0
spikeTrain_2.csv,0
spikeTrain_3.csv,1
spikeTrain_4.csv,0
spikeTrain_5.csv,2
spikeTrain_6.csv,1
</pre>


Example spik train file (Prepocessing/data/spikeTrains/1.5-SpikeTrains/spikeTrain_1.csv)
<pre>
0
0
0
.
.
.
0
0
1
0
.
.
.
</pre>

In [19]:
import pandas as pd
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

# import slayer from lava-dl
import lava.lib.dl.slayer as slayer

import IPython.display as display
from matplotlib import animation

class expermentDataloader(Dataset):
    def __init__(
        self,
        index_file: str, 
        data_path: str,
    ):
        super(expermentDataloader, self).__init__()
        self.root_dir = data_path
        self.expermentSikeTrainsIndex = pd.read_csv(index_file) # self.landmarks_frame = pd.read_csv(csv_file)
        self.inputs = [
            f"{os.path.join(self.expermentSikeTrainsIndex.iloc[i, 0])}" for i in range(len(self.expermentSikeTrainsIndex)) 
        ]
        self.targets = [
            f"{os.path.join(self.expermentSikeTrainsIndex.iloc[i, 1])}" for i in range(len(self.expermentSikeTrainsIndex)) 
        ]

    def _fileToSlayerEvents(self, fileName: str):
        CSVlines = pd.read_csv(os.path.join(self.root_dir,fileName)).to_numpy()
        events = np.array(torch.FloatTensor(CSVlines))
        
        x_event = events[:, 0]
        y_event = None
        c_event = torch.zeros(len(x_event), )
        t_event = events[:, 1]
        return slayer.io.Event(x_event,y_event,c_event,t_event)

    def __getitem__(self, index):
        input = self._fileToSlayerEvents(self.inputs[index])
        target = self._fileToSlayerEvents(self.targets[index])
        
        return (
            input.fill_tensor(torch.zeros(1, 1, 1, 3000)).squeeze(), # input spike train
            target.fill_tensor(torch.zeros(1, 1, 1, 3000)).squeeze() # target spike train
        )
        # return torch.FloatTensor(CSVlines.flatten()), int(eventClass)

    def __len__(self):
        return len(self.expermentSikeTrainsIndex)
    
indexFile5mm = "./Prepocessing/data/spikeTrainsInputTarget/5-SpikeTrains/index.csv"
PathTo5mmSpikeTrains = "./Prepocessing/data/spikeTrainsInputTarget/5-SpikeTrains"

trainingData = expermentDataloader(indexFile5mm,PathTo5mmSpikeTrains)
trainingData[0]

(tensor([0., 0., 0.,  ..., 0., 0., 0.]),
 tensor([0., 0., 0.,  ..., 0., 0., 0.]))

In [2]:
# # return Event(x_event, None, c_event, t_event / 1000)
# indexFile5mm = "./Prepocessing/data/spikeTrains/5-SpikeTrains/index.csv"
# PathTo5mmSpikeTrains = "./Prepocessing/data/spikeTrains/5-SpikeTrains"

# trainingData = expermentDataloader(indexFile5mm,PathTo5mmSpikeTrains)
# x_event = torch.zeros(len(trainingData[0][0].nonzero()[0]), )
# y_event = None
# c_event = torch.zeros(len(trainingData[0][0].nonzero()[0]), )
# t_event = np.array(trainingData[0][0].nonzero()[0])

# # x_event_2 = trainingData[0][0] * torch.zeros(len(trainingData[0][0]), )
# # y_event_2 = None
# # c_event_2 = torch.zeros(len(trainingData[0][0]), )
# # t_event_2 = np.array(list(range(len(trainingData[0][0]))))
# # print(c_event)
# slayer.io.Event(x_event,y_event,c_event,t_event).to_tensor().shape

In [3]:
indexFile5mm = "./Prepocessing/data/spikeTrains/5-SpikeTrains/index.csv"
PathTo5mmSpikeTrains = "./Prepocessing/data/spikeTrains/5-SpikeTrains"

trainingData = expermentDataloader(indexFile5mm,PathTo5mmSpikeTrains)

print(f"Is NOT all zeros: {np.any(np.array(trainingData[0][0]))}")
print(len(trainingData[0][0]))
print(f"Number of spikes: {sum(trainingData[0][0])}")
trainingData[0]


Is NOT all zeros: True
2
Number of spikes: tensor([0., 0., 0.,  ..., 0., 0., 0.])


(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 0)

In [4]:
print(f"Is NOT all zeros: {np.any(np.array(trainingData[1][0]))}")
print(len(trainingData[1][0]))
print(f"Number of spikes: {sum(trainingData[1][0])}")
trainingData[1]

Is NOT all zeros: True
2
Number of spikes: tensor([0., 0., 0.,  ..., 0., 0., 0.])


(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 1)

In [5]:
print(f"Is NOT all zeros: {np.any(np.array(trainingData[len(trainingData)-1][0]))}")
print(len(trainingData[len(trainingData)-1][0]))
print(f"Number of spikes: {sum(trainingData[len(trainingData)-1][0])}")
trainingData[len(trainingData)-1]

Is NOT all zeros: True
2
Number of spikes: tensor([0., 0., 0.,  ..., 0., 0., 0.])


(tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 2)

In [6]:
steps_per_spike_train = len(trainingData[len(trainingData)-1][0])
steps_per_spike_train

2

----
#### setup dataloader for pyTorch

In [7]:
train_loader = DataLoader(dataset=trainingData, batch_size=3)
i = iter(train_loader)
example = next(i)
type(example)

list

In [8]:
print(example[0][0].any())
print(example[1])
example = next(i)
print(example[0][0].any())
print(example[1])
example = next(i)
print(example[0][0].any())
print(example[1])
example = next(i)
print(example[0][0].any())
print(example[1])
example[0].shape

tensor(True)
tensor([0, 1, 0])
tensor(True)
tensor([2, 1, 0])
tensor(True)
tensor([2, 0, 0])
tensor(True)
tensor([0, 0, 2])


torch.Size([3, 2, 3000])

Shoul see the classes 1,0,2 and all true for any (i.e. they all have at least one event)

----
#### Building network

A `slayer.block` is a combination of `synapse`, `dendrite`, `neuron` and `axon` components. This allows for easier devlepment of SNN

In [9]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        neuron_params = {
                'threshold'     : 0.1,
                'current_decay' : 1,
                'voltage_decay' : 0.1,
                'requires_grad' : True,     
            }
        
        self.blocks = torch.nn.ModuleList([
                slayer.block.cuba.Dense(neuron_params, 2, 256),
                slayer.block.cuba.Dense(neuron_params, 256, 2),
            ])
    
    def forward(self, spike):
        for block in self.blocks:
            spike = block(spike)
        return spike

    def export_hdf5(self, filename):
        # network export to hdf5 format
        h = h5py.File(filename, 'w')
        layer = h.create_group('layer')
        for i, b in enumerate(self.blocks):
            b.export_hdf5(layer.create_group(f'{i}'))

In [10]:
trained_folder = 'Trained'
os.makedirs(trained_folder, exist_ok=True)

# device = torch.device('cpu')
device = torch.device('cuda') 

net = Network().to(device)

optimizer = torch.optim.Adam(net.parameters(), lr=0.001, weight_decay=1e-5)

We can pick between the following 
* `SpikeTime`: precise spike time based loss when target spike train is known.
* `SpikeRate`: spike rate based loss when desired rate of the output neuron is known.
* `SpikeMax`: negative log likelihood losses for classification without any rate tuning.
Sense we dont know the output spike train (as of now) and we dont have a desired spike rate we use `SpikeMax`

In [11]:
error = slayer.loss.SpikeMax().to(device)
stats = slayer.utils.LearningStats()
assistant = slayer.utils.Assistant(net, error, optimizer, stats)

In [12]:
epochs = 5000

for epoch in range(epochs):
    for i, (input, target) in enumerate(train_loader): # training loop
        output = assistant.train(input, target)
        print(f'\r[Epoch {epoch:3d}/{epochs}] {stats}', end='')
    
    if stats.training.best_loss:
        torch.save(net.state_dict(), trained_folder + '/network.pt')
    stats.update()
    stats.save(trained_folder + '/')

[Epoch   0/5000] Train loss =         inf

/home/conda/feedstock_root/build_artifacts/pytorch-recipe_1673730874951/work/aten/src/ATen/native/cuda/Loss.cu:242: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.